In [ ]:
import requests
import time
import json
import codecs
import re
import os

headers = {'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_2 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13F69 Safari/601.1',
 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
 'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', 'Accept-Encoding': 'none',
 'Accept-Language': 'en-US,en;q=0.8',
 'Connection': 'keep-alive'}

page_num = 2186

def poem_write_line(poem):
    div_start = poem.find('<div style="text-indent: -1em; padding-left: 1em;">')
    if(div_start == -1):
        return "", ""
    div_start += len('<div style="text-indent: -1em; padding-left: 1em;">')
    div_end = poem.find('</div>')
    if(div_start >= div_end):
        return "", ""
    poem_line = poem[div_start:div_end]
    poem_line = re.sub('<br>','\n',poem_line)
    poem_line = re.sub('<[^<>]*>',' ',poem_line)
    poem_line = re.sub('[ \t]+',' ',poem_line)
    poem_line = re.sub('\n+','',poem_line)
    poem_line = poem_line.strip()
    #print(poem_line)
    poem = poem[div_end+len('</div>'):]
    return poem, poem_line

def poem_crawl(link):
    
    url = link
    code = requests.get(url, headers = headers)
    text = code.text
    
    start = text.find('<div class="o-poem " data-view="PoemView">') + len('<div class="o-poem " data-view="PoemView">')
    poem = text[start:]
    poem_len = len(poem)
    
    valid = 0
    poem_data = list()
    while(poem_len != 0):
        poem, line_result = poem_write_line(poem)
        if(len(line_result) != 0):
            poem_data.append(line_result)
            valid += 1
        else:
            if(len(poem_data) == 0 or len(poem_data[-1]) != 0):
                poem_data.append(line_result)
        poem_len = len(poem)
    if(valid != 0):
        # good data
        return "\n".join(str(x) for x in poem_data)
    else:
        return ""

def poem_page_crawl(page_idx, poem_timewait):
    f = codecs.open("./data/text"+str(page_idx)+".txt", "w", "utf-8")
    f.close()
    url = "https://www.poetryfoundation.org/ajax/poems?page="+str(page_idx)+"&sort_by=recently_added"
    code = requests.get(url, headers = headers)
    text = code.text
    jsn = json.loads(text)
    entry = jsn['Entries']
    
    print(str(page_idx) + " crawling...")
    for poem in entry:
        title = poem['title']
        link = poem['link']
        data = poem_crawl(link)
        
        if(len(data) != 0):
            '''
            print('POEM_TITLE')
            print(title)
            print('POEM_DATA')
            print(data)
            '''    
            f = codecs.open("./data/text"+str(page_idx)+".txt", "a", "utf-8")
            f.write('POEM_TITLE\n')
            f.write(title + '\n')
            f.write('POEM_DATA\n')
            f.write(data + '\n')
            f.close()
            
        time.sleep(poem_timewait)
    print(str(page_idx) + " end..")
    
def crawl_all(startpoint, endpoint, list_timewait, poem_timewait):
    for i in range(startpoint, endpoint+1):
        poem_page_crawl(i, poem_timewait)
        time.sleep(list_timewait)
        
crawl_all(1, page_num, 5, 5)